![LetNet-4](image\LetNet-5.jpg)

In [1]:
#LetNet5

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO) #作用：将 TensorFlow 日志信息输出到屏幕

In [2]:
#定义模型函数
def LetNet5_model_fn(features, labels, mode):
  """Model function for LetNet4."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 6 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 6]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=6,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 6]
  # Output Tensor Shape: [batch_size, 14, 14, 6]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 16 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 6]
  # Output Tensor Shape: [batch_size, 14, 14, 16]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 16]
  # Output Tensor Shape: [batch_size, 7, 7, 16]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 16]
  # Output Tensor Shape: [batch_size, 7 * 7 * 16]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 16])

  # Dense Layer
  # Densely connected layer with 84 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 16]
  # Output Tensor Shape: [batch_size, 120]
  dense1 = tf.layers.dense(inputs=pool2_flat, units=120, activation=tf.nn.relu)

  # Dense Layer
  # Densely connected layer with 84 neurons
  # Input Tensor Shape: [batch_size, 120]
  # Output Tensor Shape: [batch_size, 84]
  dense2 = tf.layers.dense(inputs=dense1, units=84, activation=tf.nn.relu)
  

  # Add dropout operation; 0.6 probability that element will be kept
  #dropout = tf.layers.dropout(
  #    inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 84]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dense2, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  # 每500步一个Iteration
  if mode == tf.estimator.ModeKeys.TRAIN:
    boundaries = [600*2, 600*5, 600*8, 600*12]
    values = [0.01, 0.001, 0.0001, 0.00005, 0.00001]
    global_step = tf.train.get_or_create_global_step()
    learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:
#定义estimator
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  logdir = "model/mnist_LetNet5_model"
  count = "0"
  LetNet5_classifier = tf.estimator.Estimator(
      model_fn=LetNet5_model_fn, model_dir=logdir + count)

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  #tensors_to_log = {"probabilities": "softmax_tensor"}
  #logging_hook = tf.train.LoggingTensorHook(
  #    tensors=tensors_to_log, every_n_iter=500)
  
  # 运行10个Iterations
  for i in range(10):
      # Train the model
      train_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": train_data},
          y=train_labels,
          batch_size=100,
          num_epochs=None,
          shuffle=True)
      LetNet5_classifier.train(
          input_fn=train_input_fn,
          steps=600) #可加参数hooks=[logging_hook]

      # Evaluate the model and print results 
      eval_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": eval_data}, y=eval_labels, num_epochs=1, shuffle=False)
      eval_results = LetNet5_classifier.evaluate(input_fn=eval_input_fn)
      print(eval_results)


In [5]:
#运行程序
if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/mnist_LetNet5_model0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023EA91D17B8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_LetNet5_model0\model.ckpt-8400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 8400 into model/mnist_LetNet5_model0\model.ckpt.
INFO:tensorflow:loss = 0.13203254, step = 8401
INFO:tensorflow:global_step/sec: 12.2419
INFO:tensorflow:loss = 0.18470468, step = 8501 (8.169 sec)
INFO:tensorflow:global_step/sec: 13.0742
INFO:tensorflow:loss = 0.200289, step = 8601 (7.659 sec)
INFO:tensorflow:global_step/sec: 12.6695
INFO:tensorflow:loss = 0.1398847, step = 8701 (7.883 sec)
INFO:tensorflow:global_step/sec: 12.2279
INFO:tensorflow:loss = 0.135898, step = 8801 (8.178 sec)
INFO:tensorflow:global_step/sec: 9.70971
INFO:tensorflow:loss = 0.2533478, step = 8901 (10.302 sec)
INFO:tensorflow:Saving checkpoints for 9000 into model/mnist_LetNet5_model0\model.ckpt.
INFO:tensorflow:Loss for final st

INFO:tensorflow:Starting evaluation at 2019-03-05-08:40:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_LetNet5_model0\model.ckpt-11400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-05-08:40:45
INFO:tensorflow:Saving dict for global step 11400: accuracy = 0.9483, global_step = 11400, loss = 0.17474201
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 11400: model/mnist_LetNet5_model0\model.ckpt-11400
{'accuracy': 0.9483, 'loss': 0.17474201, 'global_step': 11400}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_LetNet5_model0\model.ckpt-11400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 11400 into model/mnist_Let

SystemExit: 

+ 关于`tf.app.run`有两种情况：

    + 如果你的代码中的入口函数不叫`main()`，而是一个其他名字的函数，如`test()`，则你应该这样写入口`tf.app.run(test())`
    + 如果你的代码中的入口函数叫`main()`，则你就可以把入口写成`tf.app.run()`
    
    
+ 在测试集上的准确率为94.83%，误差为5.17%，劣于论文0.95%